В этом ноуте - набросок того, как можно получать confidence score результата модели

In [10]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from torch.nn import functional as F
from typing import List, Tuple, Dict, Union


class TransformerSpellChecker:
    """
    Класс для проверки орфографии и пунктуации с использованием модели на основе Transformers.
    """

    def __init__(
        self,
        model_name: str = "ai-forever/sage-fredt5-distilled-95m",
        device: str = "cuda",
    ):
        """
        Инициализация токенизатора и модели.

        Args:
            model_name (str): Название предобученной модели.
            device (str): Устройство для вычислений ("cuda" или "cpu").
        """
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
        self.device = torch.device(device if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

    def predict_verbose(
        self, text: str
    ) -> Tuple[List[Dict[str, Union[str, float]]], str]:
        """
        Возвращает исправленный текст и список всех предложенных исправлений с оценками уверенности.

        Args:
            text (str): Входной текст для проверки.

        Returns:
            Tuple[List[Dict[str, Union[str, float]]], str]: Исправленный текст и список всех предложенных исправлений с оценками уверенности.
        """
        inputs = self.tokenizer(
            text, return_tensors="pt", truncation=False, padding=True
        )

        with torch.no_grad():
            outputs = self.model.generate(
                **inputs.to(self.device),
                max_length=inputs["input_ids"].size(1) * 1.5,
                output_scores=True,
                return_dict_in_generate=True,
            )

        # Декодируем сгенерированные токены в строки
        generated_text = self.tokenizer.batch_decode(
            outputs.sequences, skip_special_tokens=True
        )[0]

        # Генерируем список исправлений с уверенности для каждого исправления
        corrections = self._generate_corrections_with_confidence(
            text, generated_text, outputs.scores
        )

        return corrections, generated_text

    def _generate_corrections_with_confidence(
        self, original_text: str, corrected_text: str, logits: List[torch.Tensor]
    ) -> List[Dict[str, Union[str, float]]]:
        """
        Генерирует список исправлений с оценками уверенности на основе оригинального и сгенерированного текста.

        Args:
            original_text (str): Оригинальный текст.
            corrected_text (str): Исправленный текст.
            logits (List[torch.Tensor]): Логиты модели для каждого токена.

        Returns:
            List[Dict[str, Union[str, float]]]: Список исправлений с оценками уверенности.
        """
        corrections = []

        # Инициализация счетчиков для отслеживания позиции в тексте
        orig_idx = 0
        corr_idx = 0

        while orig_idx < len(original_text) and corr_idx < len(corrected_text):
            if original_text[orig_idx] == corrected_text[corr_idx]:
                # Если символы совпадают, переходим к следующему
                orig_idx += 1
                corr_idx += 1
            else:
                # Ищем исправленное слово
                orig_token_end = orig_idx
                while (
                    orig_token_end < len(original_text)
                    and original_text[orig_token_end] != " "
                ):
                    orig_token_end += 1
                original_word = original_text[orig_idx:orig_token_end]

                corr_token_end = corr_idx
                while (
                    corr_token_end < len(corrected_text)
                    and corrected_text[corr_token_end] != " "
                ):
                    corr_token_end += 1
                corrected_word = corrected_text[corr_idx:corr_token_end]

                # Получаем уверенность по логитам
                if len(logits) > corr_idx:
                    logits_for_token = logits[corr_idx]
                    probabilities = F.softmax(logits_for_token[0], dim=-1)
                    token_id = self.tokenizer.convert_tokens_to_ids(
                        self.tokenizer.tokenize(corrected_word)
                    )
                    confidence_score = probabilities[token_id[0]].item()
                else:
                    confidence_score = 0.0

                # Добавляем исправление
                corrections.append(
                    {
                        "index": orig_idx,
                        "error": original_word,
                        "suggestions": [corrected_word],
                        "confidence": confidence_score,
                        "message": f"Исправлено с вероятностью {confidence_score:.2f}",
                    }
                )

                # Продвигаем индексы
                orig_idx = orig_token_end + 1
                corr_idx = corr_token_end + 1

        return corrections

    def predict(self, text: str) -> str:
        """
        Возвращает исправленный текст.

        Args:
            text (str): Входной текст для проверки.

        Returns:
            str: Исправленный текст.
        """
        _, corrected_text = self.predict_verbose(text)
        return corrected_text

In [11]:
model = TransformerSpellChecker()

In [13]:
model.predict_verbose("превет я ондрей ,")

([{'index': 0,
   'error': 'превет',
   'suggestions': ['Привет,'],
   'confidence': 0.9987919926643372,
   'message': 'Исправлено с вероятностью 1.00'},
  {'index': 9,
   'error': 'ондрей',
   'suggestions': ['Андрей.'],
   'confidence': 0.0,
   'message': 'Исправлено с вероятностью 0.00'}],
 'Привет, я Андрей.')